In [2]:
import os
import time
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

### Data Collection:

* [Shakespeare dataset](https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt)
* [Shakespeare dataset](https://raw.githubusercontent.com/reddyprasade/Deep-Learning-with-Tensorflow-2.x/main/Data/input.txt)

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [6]:
# Read the text file data
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
print("Length of text:{} charaters".format(len(text)))

Length of text:1115394 charaters


In [8]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [10]:
vocab = sorted(set(text))
print("{} Unique Characters From Text Data".format(len(vocab)))

65 Unique Characters From Text Data


In [11]:
example_text = ['abcdefg','xyz']

In [12]:
chars = tf.strings.unicode_split(example_text,input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

In [14]:
ids_from_chars

In [15]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>

In [16]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [17]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [18]:
tf.strings.reduce_join(chars,axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [19]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [20]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([20, 49, 58, ..., 47, 10,  2])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [22]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [39]:
len(text)

1115394

In [25]:
seq_length = 100
example_per_epoch =len(text)//(seq_length+1)

In [27]:
sequences = ids_dataset.batch(seq_length+1,drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [28]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [30]:
def split_input_target(sequences):
  input_text = sequences[:-1]
  target_text = sequences[1:]
  return input_text,target_text


In [33]:
split_input_target(list('Python'))

(['P', 'y', 't', 'h', 'o'], ['y', 't', 'h', 'o', 'n'])

In [34]:
dataset = sequences.map(split_input_target)

In [35]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [36]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000


In [37]:
dataset = (dataset
           .shuffle(BUFFER_SIZE)
           .batch(BATCH_SIZE,drop_remainder=True)
           .prefetch(tf.data.experimental.AUTOTUNE)
           )

In [38]:
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [51]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [59]:
class MyModel(tf.keras.Model):
  
  
  def __init__(self,vocab_size,embedding_dim,rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
  


  def call(self,inputs,states=None,return_state=False,training=False):
    x = inputs
    x = self.embedding(x,training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x,states = self.gru(x, initial_state=states,training=training)
    x = self.dense(x,training=training)
    if return_state:
      return x,states
    else:
      return x

In [60]:
model = MyModel(vocab_size= len(ids_from_chars.get_vocabulary()),
                embedding_dim=embedding_dim,
                rnn_units = rnn_units
    
)

In [62]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions  = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 67)


* **64** is my batch_size
* **100** is my sequence_length
* **67** is my vocab_size

In [63]:
model.summary()

Model: "my_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  17152     
_________________________________________________________________
gru_2 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_2 (Dense)              multiple                  68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [64]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [65]:
sampled_indices

array([14, 44, 16,  7, 13, 13, 59, 40, 41, 48,  4,  8, 14, 24, 56, 37,  2,
       64, 13, 63,  0, 25, 18, 21, 56,  8, 39, 23, 21, 37, 50,  6, 38, 32,
       33, 53, 53, 54, 35, 45, 61, 28, 15, 20,  0,  7, 30, 55,  4, 45, 11,
       47, 52, 57,  2, 27,  8, 30, 24, 47, 46, 47, 32, 21, 24,  8, 42,  2,
        9, 51, 52, 40, 54, 15, 44, 21, 14,  9,  7, 13, 63, 47, 36, 22, 34,
        8, 29, 24, 41, 61,  7,  3, 59, 28, 17, 59, 26, 17, 36, 33])

In [66]:
print("Input \n",text_from_ids(input_example_batch[0].numpy()))

Input 
 tf.Tensor(b'II:\nBarkloughly castle call they this at hand?\n\nDUKE OF AUMERLE:\nYea, my lord. How brooks your grace', shape=(), dtype=string)


In [68]:
print("Next Char Predication:\n",text_from_ids(sampled_indices).numpy())

Next Char Predication:
 b"?dB';;sZah!,?JpW\nx;wKDGp,YIGWj&XRSmmnUeuNAF'Po!e3glq\nM,PJgfgRGJ,b\n-klZnAdG?-';wgVHT,OJau' sNCsLCVS"


In [69]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [70]:
example_batch_loss  = loss(target_example_batch,example_batch_predications)
mean_loss = example_batch_loss.numpy().mean()

In [71]:
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.205204


In [72]:
tf.exp(mean_loss).numpy()

67.03427

In [73]:
model.compile(optimizer='adam',loss=loss)

In [74]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [77]:
EPOCHS = 5

In [ ]:
history = model.fit(dataset,epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

Epoch 1/5
 30/172 [====>.........................] - ETA: 12:28 - loss: 2.7559

### Generate text